# Crear bolsa de fichas y visualizar las fichas de la bolsa

In [1]:
from ApylabradosModule import Pawns

my_pawns = Pawns()
my_pawns.createBag()
my_pawns.showPawns()

Numero total de fichas: 100
{'A': 12, 'B': 2, 'C': 4, 'D': 5, 'E': 12, 'F': 2, 'G': 2, 'H': 2, 'I': 6, 'J': 1, 'K': 1, 'L': 4, 'M': 3, 'N': 5, 'O': 9, 'P': 2, 'Q': 1, 'R': 5, 'S': 7, 'T': 4, 'U': 6, 'V': 1, 'W': 1, 'X': 1, 'Y': 1, 'Z': 1}


In [2]:
for pawn in range(7):
    my_pawns.takeRandomPawn()
my_pawns.showPawns()    

has agarrado la letra: D
Estas son mis letras:
['D']
has agarrado la letra: R
Estas son mis letras:
['R']
has agarrado la letra: A
Estas son mis letras:
['A']
has agarrado la letra: N
Estas son mis letras:
['N']
has agarrado la letra: U
Estas son mis letras:
['U']
has agarrado la letra: D
Estas son mis letras:
['D']
has agarrado la letra: C
Estas son mis letras:
['C']
Numero total de fichas: 93
{'A': 11, 'B': 2, 'C': 3, 'D': 3, 'E': 12, 'F': 2, 'G': 2, 'H': 2, 'I': 6, 'J': 1, 'K': 1, 'L': 4, 'M': 3, 'N': 4, 'O': 9, 'P': 2, 'Q': 1, 'R': 4, 'S': 7, 'T': 4, 'U': 5, 'V': 1, 'W': 1, 'X': 1, 'Y': 1, 'Z': 1}
